In [1]:
# import all you need
import numpy as np
import plotly.graph_objects as go
from scipy.optimize import curve_fit

# functions imported from helper_files
from helper_files.plotting import plot_lines, plotly_plot
from helper_files.gaussian_fitting import gaussian, n_gaussians, fit_n_peaks_to_gaussian
from helper_files.read_data import read_xy_data, read_only_y_data
from helper_files.error_calculation import rms_error
from helper_files.calibration import calibrate_channel_width_two_peaks

In [2]:
# this will load the helper modules each time you make changes to them, without having to restart the kernel
%load_ext autoreload
%autoreload 2 # or 1?????

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Goal of this notebook:

Use a spectrum with known peaks to calibrate the channels of a detector,
and then use the calibration to plot an unknown spectrum.

To calibrate a spectrum we calculate:
  - dispersion [keV / channel], which is the width of each channel
  - offset [channels], which is the distance from first data entry to keV=0

When we have calibrated a spectrum, we could also find:
  - resolution,
  - quantities of each element, 
  - ... ?


### What we will do:

1. Read in the data of a known spectrum
2. Fit the data to a gaussian
3. Calibrate the x-axis
4. Plot the calibrated data
5. Use the calibrated x-axis on an unknown spectrum from the same source



In [3]:
# the '*' argument is that all arguments (after) must be named (kwarg) and not just positional arguments
def init_known_spectrum(
    *,
    name,
    filepath,
    start_str,
    stop_str,
    line_endings,
    delimiter=None,
    peaks_keV=None,
    peaks_names=None,
    peaks_channel=None,
    plot_title="unnamed plot"
):
    # if the file is with x and y, delimiter must be specified
    if delimiter:
        data_raw = read_xy_data(filepath, start_str, stop_str, delimiter, line_endings)
        keV_uncalibrated = data_raw[0]
    else:
        data_raw = read_only_y_data(filepath, start_str, stop_str, line_endings)
        keV_uncalibrated = None
    if data_raw is None:
        print(
            "ERROR: could not read the data properly, please check the parameters for reading the file"
        )
        return None

    # this is the data set we will be working with.
    #       - y: counts normalized to the maximum peak
    #       - x: channels
    intensity = data_raw[1] / data_raw[1].max()
    channels = np.arange(0, len(intensity), 1)

    # making the dictionary we will work with
    spectrum = {
        "name": name,
        "channel": channels,
        "intensity": intensity,
        "peaks_keV": peaks_keV,
        "peaks_names": peaks_names,
        "peaks_channel": peaks_channel,
        "plot_title": plot_title,
        "dispersion": None,
        "offset": None,
        "kev_calibrated": None,
        "fit_params": None,
        "fit_cov": None,
        "intensity_fit": None,
        "rms_error": None,
    }
    return spectrum

### Reading the data into arrays

##### See the helper file plotting.py

Here you need to:

- set the path to the file, or just use the example file
- open the data file and find out where the data starts and stops.
    - in the example data the data starts after "#SPECTRUM    : Spectral Data Starts Here",
    - and ends with "#ENDOFDATA   : "
- take note of the line endings,
    - in the example data the line endings are '\n'
- take note of what the data is separated by, 
    - in the example data it is separated by a comma and a space ", "

We use these variables to read the data with either:

- read_xy_data(...) if the file contains both x and y on each line
- read_only_y_data(...) if the file only contains the counts

In [4]:
# reading the file

# reading the .emsa file
s_GaAs_emsa = init_known_spectrum(
    name="GaAs 30kV from .emsa",
    filepath="Ex1_EDS_GaAs_30kV.emsa",
    start_str="#SPECTRUM    : Spectral Data Starts Here",
    stop_str="#ENDOFDATA   : ",
    line_endings="\n",
    delimiter=", ",
    peaks_keV=[1.098, 9.2517],
    peaks_names=["Ga_La", " Ga_Ka"],
    peaks_channel=[130, 944, 22, 46, 117,  149, 195,  1045, 1072, 1191],
    plot_title="GaAs 30kV from .emsa",
)
# # # Ex1, As peaks, to compare the result
# # peak1_As = [149.403,  1.2819]
# # peak2_As = [1072.334, 10.5436]

# # Ex2 .msa
# filepath = 'Ex2_NiO_on_Mo_not_calibrated.msa'
# start_string = "#SPECTRUM    : Spectral Data Starts Here"
# stop_string = "#ENDOFDATA   : End Of Data and File"
# line_endings = ', \n'
# data = read_only_y_data(filepath, start_string, stop_string, line_endings)
# # peak1 = [766.522, 7.478]
# # peak2 = [105.533, 0.8515]

# # Ex3 .mca
# filepath = 'Ex3_Cu.mca'
# start_string = "<<DATA>>"
# stop_string = "<<END>>"
# line_endings = '\n'
# data = read_only_y_data(filepath, start_string, stop_string, line_endings)


# # unknown_mse
# filepath = 'example_data/unknown_not_calibrated.msa'
# start_string = "#SPECTRUM    : Spectral Data Starts Here"
# stop_string = "#ENDOFDATA   : End Of Data and File"
# line_endings = ', \n'
# data = read_only_y_data(filepath, start_string, stop_string, line_endings)
# # peak1 = [380.699, 3.595]
# # peak2 = [943.75, 9.2517]


The first line looks like this: '#FORMAT      : EMSA/MAS Spectral Data File\n'
Reading from line 42 to 2091.
Read 2048 data points from Ex1_EDS_GaAs_30kV.emsa
First entry: [-0.2, 0.0]
Last entry: [20.27, 52.0]


In [5]:
s = s_GaAs_emsa
# s = s_GaAs_emsa.copy()  # only necessary if you want to work with a copy of the spectrum

In [6]:
fig_intitial = plotly_plot(
    x=s["channel"],
    y=s["intensity"],
    title=s["plot_title"],
    xaxis_title="channel",
    yaxis_title="intensity",
)
fig_intitial.show()

### Fittin the peaks to gaussians

##### see the helper file gaussian_fitting.py

Now we find two or more peaks in the data, and we want to fit a gaussian to each of them. 
We need to know what the theoretical value of at least two the guessed peaks, so that we can calibrate the spectrum later.

In the Ex1 we can use Ga_Ka=9.2517 keV and Ga_La=1.098 keV, or we can use As_Ka=10.5336 keV and As_La=1.2819 keV. Or we can use both

eg peak_guesses = [9.2517, 10.5336]

Short overview of the used functions in gaussian_fitting.py
- def gaussian(x, amp, mu, sigma):
    - the function gaussian defines a gaussian function.

- def n_gaussians(x, *args):
    - since the gaussians could potentially partially overlap, we need to define a function that returns the sum of n gaussians.
        - Eg. like Ga_Kb=10.2642 and As=10.5436 in Ex1

- def fit_n_peaks_to_gaussian(x, raw_y, guessed_peaks, guessed_std=1, guessed_amp=1,):
    - now we need a functions which fits peak guesses to gaussian curves.
    - we will use the scipy.optimize.curve_fit function for this. More info in the function.
    - with normalized counts it usually works nice with guessing all std and amplitues as 1

Additional info:
I tried fitting to the raw keV-valus of the .emsa file, but that did not work. 
I think it is because the x values are not integers, and for some reason that hindered the fitting.
I do suspect that it migh be because the std and amp guesses are way off, but I am not sure.

This is the code that did not work:
~~~
peak_guesses = [0.02, 0.27, 0.97, 1.1, 1.29, 1.75, 9.2517, 10.24, 10.5336, 11.75] 
fit_vals = fit_n_peaks_to_gaussian(data[0], data[1], peak_guesses)
~~~

**Be aware: sometimes the fitting sets a peak as the background. Always inspect the plot**

*TODO: implement RMS-errors between the data and the fit*


In [7]:
print(f'Check if the peaks are in the right place in the plot above: {s["peaks_channel"]}')
print('The two first peaks are the ones we will use for calibration later, ie. the ones we know the energy of.')

Check if the peaks are in the right place in the plot above: [130, 944, 22, 46, 117, 149, 195, 1045, 1072, 1191]
The two first peaks are the ones we will use for calibration later, ie. the ones we know the energy of.


In [8]:
# Fitting the data to gaussians

#
#
# NB! if this cell crash with:
# "RuntimeError: Optimal parameters not found: Number of calls to function has reached maxfev = 5000."
# then adjust your peak guesses, or try to fit fewer peaks at once.
#
#

# # Eg. for GaAs
# s['peaks_channel'] = [130, 944, 22, 46, 117, 149, 195, 1045, 1072, 1191]

if s["peaks_channel"] is None:
    print("ERROR: no peaks specified, please specify the peaks in the spectrum. No fitting was done.")
    print("Comment out the line above to set s['peaks_channel'] =[peak1, peak2, ...] manually.")
else:
    fit_vals = fit_n_peaks_to_gaussian(
        x=s["channel"],
        y=s["intensity"],
        guessed_peaks=s["peaks_channel"],
    )
    s["fit_params"] = fit_vals[0]
    s["fit_cov"] = fit_vals[1]
    
    # update the channel values of the peaks
    s["peaks_channel"] = s["fit_params"][1::3]

    # addin the fitted gaussian to the spectrum-dictionary
    s['intensity_fit'] = n_gaussians(s['channel'], *fit_vals[0])


print(f'Fitted peaks at: {s["peaks_channel"]}')

Fitted peaks at: [ 130.59922763  943.74965817   21.89794811   46.72482409  116.95113395
  149.40335628  471.15575632 1045.74673278 1072.33433649 1191.41060414]


In [47]:
for key, val in s.items():
    print(f'{key}')

name
channel
intensity
peaks_keV
peaks_names
peaks_channel
plot_title
dispersion
offset
kev_calibrated
fit_params
fit_cov
intensity_fit
rms_error


In [53]:
# plot using the following function, see helper_files/plotting.py

fig_fit = plotly_plot(
    s['channel'], s['intensity'], y_fit=s['intensity_fit'], vlines=s['peaks_channel'], vlines_name=s['peaks_names'],  title=f"Gaussian fitting of {s['plot_title']}",
)
# comment in the line below to add the fit of each peak as its own line
# fig_fit = plotly_plot(x=s['channel'], fig=fig_fit, fit_params=s['fit_params'])  
fig_fit.show()

In [37]:
# now we use the fitted peak center and the theoretical value

# it is important that the two first peaks in s['peaks_channel'] are corresponding to the two first peaks in s['peaks_keV']

# # GaAs
# s['peaks_keV'] = [1.098, 9.2517]

if (s['peaks_keV'] is None):
    print('ERROR: no peaks specified, please specify the peaks in the spectrum. No calibration was done.')
    print("Comment out the line above to set s['peaks_keV'] =[peak1, peak2] manually.")
else:
    calib = calibrate_channel_width_two_peaks(s['peaks_channel'], s['peaks_keV'])
    s['dispersion'] = calib[0]
    s['offset'] = calib[1]
    s['kev_calibrated'] = (s['channel'] - s['offset']) * s['dispersion']

The calibration factor is: 0.0100273 keV/channel, with 21.098 channels zero offset


In [42]:
# plotting the calibrated data

fig_calib = plotly_plot(
    x=s['kev_calibrated'],
    y_named=[s['intensity'], "x calibrated with Ga_Ka and Ga_La"],
    vlines=s['peaks_keV'],
    vlines_name=s['peaks_names'],
    title=f"Calibrated {s['plot_title']}",
) 

fig_calib.show()

### Error calculation

##### See the helper file error_calculation.py

We want to know the error of the fitting. For now RMS-error is implemented.

- rms_error(s)
    - calculates the RMS-error between s['intensity'] and s['intensity_fit']
- Good values are ___?


TODO: RMSE are now calculating on the whole fit, not just on/around the peaks

In [15]:
rmse = rms_error(s['intensity'], s['intensity_fit'])
s['rms_error'] = rmse
print(f"Root mean square error: {rmse}")

Root mean square error: 0.0020354573184980518


In [14]:
s

{'name': 'GaAs 30kV from .emsa',
 'channel': array([   0,    1,    2, ..., 2045, 2046, 2047]),
 'intensity': array([0.        , 0.        , 0.        , ..., 0.00078112, 0.00085086,
        0.00072533]),
 'peaks_keV': [1.098, 9.2517],
 'peaks_names': ['Ga_La', ' Ga_Ka'],
 'peaks_channel': array([ 130.59922763,  943.74965817,   21.89794811,   46.72482409,
         116.95113395,  149.40335628,  471.15575632, 1045.74673278,
        1072.33433649, 1191.41060414]),
 'plot_title': 'GaAs 30kV from .emsa',
 'dispersion': 0.010027295926808828,
 'offset': 21.09812104823166,
 'kev_calibrated': array([-0.2115571 , -0.20152981, -0.19150251, ..., 20.29426307,
        20.30429036, 20.31431766]),
 'fit_params': array([9.96111646e-01, 1.30599228e+02, 3.06246253e+00, 3.50060406e-01,
        9.43749658e+02, 6.66879787e+00, 1.07747996e-01, 2.18979481e+01,
        1.70951158e+00, 1.21687946e-02, 4.67248241e+01, 1.41537931e+00,
        6.67361231e-02, 1.16951134e+02, 4.67510721e+00, 3.90884577e-01,
        1